In [ ]:
import numpy as np
import torch
import torch.nn as nn

import neuralpde

# RK4 integration scheme
rk4_A = torch.tensor(
    [
        [  0,   0,   0,   0],
        [1/2,   0,   0,   0],
        [  0, 1/2,   0,   0],
        [  0,   0,   1,   0]
    ],
    requires_grad=False
)
rk4_b = torch.tensor([1/6, 1/3, 1/3, 1/6], requires_grad=False)
rk4_c = torch.tensor([0, 1/2, 1/2, 1], requires_grad=False)
# this is almost certainly not sufficient (not enough steps.  It's also an explicit method)
# I will implement using a good enough RK scheme (i.e., any of those from Raissi's codebase)
# sometime later

# hyperparameters (i.e., relative weighting of terms in loss)
weights = torch.tensor(
    [
        1.,  # differential loss (t_{n})
        1.,  # differential loss (t_{n+1})
        1.,  # boundary loss, no slip
        1.,  # boundary loss, no penetration
        1.,  # kappa regularization
        1.,  # v regularization
        1.,  # f regularization
    ],
    requires_grad=False
)
weights = weights / torch.sqrt(torch.sum(weights**2))

# dt = 1, right?
dt = 1


Note that the PDE,
$$
u_t = \nabla \cdot (\kappa \nabla u) + v \cdot \nabla u + f
$$
can be equjvalently written as,
$$
\begin{aligned}
u_t &= \kappa \Delta u + (\nabla \kappa) \cdot (\nabla u) + v \cdot \nabla u + f \\
&= \kappa (u_{xx} + u_{yy}) + (\kappa_x u_x + \kappa_y u_y) + (v_1 u_x + v_2 u_y) + f.
\end{aligned}
$$

In [ ]:
def loss(net: neuralpde.network.Network, x: torch.tensor, uj: torch.tensor):
    """
    Loss function to train convolutional network to extract PDE parameters.

    Arguments:
        net:        Network being optimized.
        x:          Spatial coordinate pairs like of the shape `(2, N, M)`.
        uj:         Intermediate RK stage estimates produced from network in the shape `(q, N, M)`.

    Other important symbols in this function:
        uhat_i:     Approximation to u_{n} computed from intermediate RK stages uj.
        uhat_f:     Approximation to u_{n+1} computed from intermediate RK stages uj.
    """
    uj_x, uj_y = torch.autograd.grad(uj, x, torch.ones_like(uj), create_graph=True)
    uj_xx = torch.autograd.grad(uj_x, x[0], torch.ones_like(uj))[-1]
    uj_yy = torch.autograd.grad(uj_y, x[1], torch.ones_like(uj))[-1]
    kappa_x, kappa_y = torch.autograd.grad(net.kappa, x, torch.ones_like(net.kappa))
    v_x, v_y = torch.autograd.grad(net.v, x, torch.ones_like(net.v))

    # _N like (u_{n+c1}, u_{n+c2}, ...)
    _N = net.kappa * (uj_xx + uj_yy) + (kappa_x * uj_x + kappa_y * kappa_y) + \
        (net.v[0] * uj_x + net.v[1] * uj_y) + net.f

    # FIXME: this matmul might be wrong, in which case I need to refigure what it *should* be
    # if this causes problems, also consider that this (eq. (22) in Raissi 2019) might be wrong
    # eq. (22) in the paper differs from this line in his code
    # https://github.com/maziarraissi/PINNs/blob/1b3e90e82c47d49aee290aa481550f5e9c582d9a/main/discrete_time_identification%20(KdV)/KdV.py#L130
    uhat_i = uj + dt * torch.matmul(_N, rk4_A.T)
    # uhat_i = uj - dt * torch.matmul(_N, rk4_A.T)
    uhat_f = uj + dt * torch.matmul(_N, (rk4_A - rk4_b).T)
    
    # loss terms that still need to be added:
    # differential loss (t_{n})
    # differential loss (t_{n+1})
    # boundary loss, no slip
    # boundary loss, no penetration
    # kappa regularization
    # v regularization
    # f regularization
